# Required library

In [1]:
import pandas as pd, numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from pathlib import Path
%matplotlib inline

In [2]:
import itertools as it
from pathlib import Path

pd.options.display.max_columns=305

In [3]:
import lightgbm  as lgb
import gc
from sklearn.model_selection import train_test_split
import time
from tqdm.notebook import tqdm

import re,json

# Reading the data

In [4]:
DATA_ROOT = Path("../input/lyft-train-as-parquet/train")

In [5]:
def get_scene_path(scene):
    meta = "meta_{}_{}.json".format(*re.search( r"scenes_(\d+)_(\d+)", scene.stem).groups())
    with open(DATA_ROOT/meta) as f:
        meta = json.load(f)
    frame = DATA_ROOT/meta["frames"]["results"]["filename"]
    agent = DATA_ROOT/meta["agents"]["results"]["filename"]
    return (scene, frame, agent)

In [6]:
SCENES = np.array(list(DATA_ROOT.glob("scenes_*.parquet.snappy")))
SCENES = SCENES[np.random.permutation(len(SCENES))]
print("NB SCENES:", len(SCENES))
scene = SCENES[0]
scene

NB SCENES: 33


PosixPath('../input/lyft-train-as-parquet/train/scenes_000500_001000_500.parquet.snappy')

In [7]:
get_scene_path(scene)

(PosixPath('../input/lyft-train-as-parquet/train/scenes_000500_001000_500.parquet.snappy'),
 PosixPath('../input/lyft-train-as-parquet/train/frames_00124167_00248301_124134.parquet.snappy'),
 PosixPath('../input/lyft-train-as-parquet/train/agents_0010019001_0020042294_10023293.parquet.snappy'))

In [8]:
reader = pd.read_parquet

In [9]:
def merge(scenes, frames, agents, shift, verbose=False):
    df = scenes.merge(frames, on = "scene_db_id")
    df = df.merge(agents, on="frame_db_id")

    shift_cols = [
            "centroid_x", "centroid_y",
            "yaw",
            "velocity_x","velocity_y",
            "nagents","nlights",
            'extent_x', 'extent_y','extent_z',
            'label_probabilities_PERCEPTION_LABEL_UNKNOWN',
            'label_probabilities_PERCEPTION_LABEL_CAR', 
            'label_probabilities_PERCEPTION_LABEL_CYCLIST', 
            'label_probabilities_PERCEPTION_LABEL_PEDESTRIAN',
    ]
    new_shift_cols = ["centroid_xs", "centroid_ys"] + shift_cols[2:]

    df[new_shift_cols] = df.groupby(["scene_db_id", "track_id"])[shift_cols].shift(shift)
    nulls = df[["centroid_xs", "centroid_ys", "yaw", "velocity_x","velocity_y"]].isnull().any(1)
    shape0 =  df.shape
    df = df[~nulls]
    
    if verbose:
        print("SHAPE0:", shape0)
        print("nulls ratio:", nulls.sum()/shape0[0])
    
    return df

In [10]:
def read_all(shift=1, max_len=1e5):
    """
    Read parquet files from the SCENES list until the df's size is greater than `max_len`.
    
    If you want better accuracy, you need to increase `max_len`.
    With `max_len=12e6` I got a score of 200.xxx.
    But note that training time increases as max_len increases.
    """
    dfs = None
    scenes = SCENES[np.random.permutation(len(SCENES))]
    for scene in scenes:
        SCENES_FILE,FRAMES_FILE,AGENTS_FILE = get_scene_path(scene)
        
        scenes = reader(SCENES_FILE)
        
        frames = reader(FRAMES_FILE)
        frames["nagents"] = frames["agent_index_interval_end"] - frames["agent_index_interval_start"]
        frames["nlights"] = frames["traffic_light_faces_index_interval_end"
                                  ] - frames["traffic_light_faces_index_interval_start"]
    
        agents = reader(AGENTS_FILE)
        agents.rename(columns = {"agent_id": "agent_db_id"}, inplace=True)
        
        df = merge(scenes, frames, agents, shift=shift)
        
        dfs = df if dfs is None else dfs.append(df)
        dfs.reset_index(inplace=True, drop=True)
        
        if len(dfs) > max_len:
            break
    
    return dfs

# LGBM trainer

In [11]:
def lgbm_trainer(shift=1, root=None, params=None):
    t0 = time.strftime("%Y%m%d%H%M%S")
    T00 = time.time()
    root = "model_{}".format(t0) if root is None else str(root)
    params = PARAMS if params is None else params
    
    df = read_all(shift=shift)
    print("df.shape:", df.shape)
    
    df_centroid = df[["centroid_x", "centroid_y"]]
    df = df[TRAIN_COLS]
    gc.collect()
    
    train_index, test_index = train_test_split(df.index.values.reshape((-1,1)),
                                           df.index.values, test_size = .20, random_state=177)[2:]
    print("\n")
    for suffix in ["x", "y"]:
        print("--> {}".format(suffix.upper()))
        target_name = "centroid_" + suffix
        target = (df_centroid[target_name] - df[target_name+"s"])
    
        train_data = lgb.Dataset(df.loc[train_index], label= target.loc[train_index])
        test_data = lgb.Dataset(df.loc[test_index], label= target.loc[test_index])

        clf = lgb.train(params,
                        train_data,
                        valid_sets = [train_data, test_data],
                        early_stopping_rounds=60, 
                        verbose_eval= 40
                       )
        
        clf.save_model("models/{}/lgbm_{}_shift_{:02d}.bin".format(root, suffix, shift))
        print('\n')
    print("elapsed: {:.5f} min".format((time.time()-T00)/60))

In [12]:
def get_time(format_="%Y-%m-%d %H:%M:%S"):
    return time.strftime(format_)

In [13]:
def train_50_shifts(root=None):
    root = root or  "model_{}".format(time.strftime("%Y%m%d%H%M%S"))
    Path("models").joinpath(root).mkdir(exist_ok=True, parents=True)
    params = PARAMS.copy()
    for shift in tqdm(list(range(50, 0, -1))):
        print('\n ******************* SHIFT {:02d} {} ***********\n'.format(shift,get_time()))
        
        if not (shift-1)%5:
            params["num_iterations"] = max(100, params["num_iterations"] - 20)
            params["num_leaves"] = max(31, params["num_leaves"] - 10)
        
        meta = {
            "TRAIN_COLS": TRAIN_COLS,
            "params": params,
            "shift": shift,
            "start": get_time(),
            "end": None
        }
        
            
        lgbm_trainer(root=root, shift=shift, params=params)
        meta["end"] = get_time()
        with open("models/{}/meta_shift_{:02d}.json".format(root, shift), "w") as f:
            json.dump(meta, f, indent=2)

# Params

**I fixed this phase.**

In [14]:
PARAMS = {
         'objective':'regression',
         'boosting': 'gbdt',
         'feature_fraction': 0.5 ,
         'scale_pos_weight' : 1/40., 
         'num_iterations' : 200,
         'learning_rate' :  0.7,
         'max_depth': 41,
         'min_data_in_leaf': 64,
         'num_leaves': 128,
         'bagging_freq' : 1,
         'bagging_fraction' : 0.8,
         'tree_learner': 'voting',
         'boost_from_average': True,
         'verbosity' : 0,
         'num_threads': 2,
         'metric' : ['mse'],
         'metric': [ "l1", "rmse"],
         "verbosity": 1,
         'reg_alpha': 0.1,
         'reg_lambda': 0.3
        }

In [15]:
# Uncomment the columns if you want more

TRAIN_COLS = [
     'ego_translation_x', 
     'ego_translation_y', 
     'ego_translation_z', 
     'ego_rotation_xx', 
     'ego_rotation_xy', 
     'ego_rotation_xz', 
     'ego_rotation_yx', 
     'ego_rotation_yy', 
     'ego_rotation_yz', 
     'ego_rotation_zx', 
     'ego_rotation_zy', 
     'ego_rotation_zz', 
    'extent_x', 
    'extent_y', 
    'extent_z', 
    'velocity_x', 
    'velocity_y', 
    'label_probabilities_PERCEPTION_LABEL_UNKNOWN', 
    'label_probabilities_PERCEPTION_LABEL_CAR', 
    'label_probabilities_PERCEPTION_LABEL_CYCLIST', 
    'label_probabilities_PERCEPTION_LABEL_PEDESTRIAN', 
    'yaw', 
    'nagents', 
    'nlights', 
    'centroid_xs', 
    'centroid_ys',
]

In [16]:
print("len(TRAIN_COLS):", len(TRAIN_COLS))

len(TRAIN_COLS): 26


# Training

In [17]:
%%time

# Train 50x2 lgbm models (50 time dimensions X 2 space dimensions)
# Save it as lgbm_{x or y}_shift_{i:02d}
# Each model has it's own meta_shift_{i:02d} which contains the model's params
# You can juts feed the ouputs as inputs to the inference kernel
# The inference kernel is at https://www.kaggle.com/kneroma/lgbm-on-lyft-tabular-data-inference

train_50_shifts("lyft_lgbm_model")


 ******************* SHIFT 50 2020-09-10 00:53:26 ***********

df.shape: (2471188, 42)


--> X


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 60 rounds
[40]	training's l1: 1.07943	training's rmse: 2.90353	valid_1's l1: 1.11335	valid_1's rmse: 3.01955
[80]	training's l1: 0.985603	training's rmse: 2.61322	valid_1's l1: 1.03978	valid_1's rmse: 2.79593
[120]	training's l1: 0.934208	training's rmse: 2.41761	valid_1's l1: 1.00404	valid_1's rmse: 2.65498
[160]	training's l1: 0.895033	training's rmse: 2.28991	valid_1's l1: 0.978148	valid_1's rmse: 2.561
[200]	training's l1: 0.870624	training's rmse: 2.18883	valid_1's l1: 0.965831	valid_1's rmse: 2.50831
Did not meet early stopping. Best iteration is:
[200]	training's l1: 0.870624	training's rmse: 2.18883	valid_1's l1: 0.965831	valid_1's rmse: 2.50831


--> Y
Training until validation scores don't improve for 60 rounds
[40]	training's l1: 1.22952	training's rmse: 3.32966	valid_1's l1: 1.27323	valid_1's rmse: 3.46945
[80]	training's l1: 1.11372	training's rmse: 2.92384	valid_1's l1: 1.18006	valid_1's rmse: 3.1379
[120]	training's l1: 

[40]	training's l1: 1.07837	training's rmse: 2.95606	valid_1's l1: 1.11205	valid_1's rmse: 3.08554
[80]	training's l1: 0.975647	training's rmse: 2.64039	valid_1's l1: 1.02589	valid_1's rmse: 2.83151
[120]	training's l1: 0.926137	training's rmse: 2.45303	valid_1's l1: 0.990443	valid_1's rmse: 2.6957
[160]	training's l1: 0.890204	training's rmse: 2.33053	valid_1's l1: 0.965535	valid_1's rmse: 2.60766
Did not meet early stopping. Best iteration is:
[180]	training's l1: 0.876156	training's rmse: 2.27435	valid_1's l1: 0.957669	valid_1's rmse: 2.57028


elapsed: 1.94163 min

 ******************* SHIFT 44 2020-09-10 01:04:55 ***********

df.shape: (2575502, 42)


--> X
Training until validation scores don't improve for 60 rounds
[40]	training's l1: 0.924798	training's rmse: 2.48672	valid_1's l1: 0.960232	valid_1's rmse: 2.62063
[80]	training's l1: 0.858858	training's rmse: 2.25912	valid_1's l1: 0.90857	valid_1's rmse: 2.45073
[120]	training's l1: 0.818206	training's rmse: 2.0883	valid_1's l1:

[80]	training's l1: 0.841421	training's rmse: 2.28047	valid_1's l1: 0.881937	valid_1's rmse: 2.44195
[120]	training's l1: 0.801813	training's rmse: 2.15477	valid_1's l1: 0.852689	valid_1's rmse: 2.34936
[160]	training's l1: 0.776653	training's rmse: 2.06096	valid_1's l1: 0.835878	valid_1's rmse: 2.28387
Did not meet early stopping. Best iteration is:
[160]	training's l1: 0.776653	training's rmse: 2.06096	valid_1's l1: 0.835878	valid_1's rmse: 2.28387


elapsed: 2.00380 min

 ******************* SHIFT 38 2020-09-10 01:16:48 ***********

df.shape: (3213335, 42)


--> X
Training until validation scores don't improve for 60 rounds
[40]	training's l1: 0.706112	training's rmse: 1.98216	valid_1's l1: 0.725745	valid_1's rmse: 2.06991
[80]	training's l1: 0.651132	training's rmse: 1.80335	valid_1's l1: 0.681357	valid_1's rmse: 1.93453
[120]	training's l1: 0.622435	training's rmse: 1.70492	valid_1's l1: 0.661221	valid_1's rmse: 1.86741
[160]	training's l1: 0.602493	training's rmse: 1.62815	valid_

Did not meet early stopping. Best iteration is:
[140]	training's l1: 0.571142	training's rmse: 1.52072	valid_1's l1: 0.605089	valid_1's rmse: 1.66715


--> Y
Training until validation scores don't improve for 60 rounds
[40]	training's l1: 0.741816	training's rmse: 2.07502	valid_1's l1: 0.761184	valid_1's rmse: 2.16063
[80]	training's l1: 0.687281	training's rmse: 1.88116	valid_1's l1: 0.714152	valid_1's rmse: 1.9969
[120]	training's l1: 0.657753	training's rmse: 1.77633	valid_1's l1: 0.692263	valid_1's rmse: 1.9188
Did not meet early stopping. Best iteration is:
[140]	training's l1: 0.647908	training's rmse: 1.74177	valid_1's l1: 0.684998	valid_1's rmse: 1.89303


elapsed: 1.99705 min

 ******************* SHIFT 31 2020-09-10 01:30:24 ***********

df.shape: (3524461, 42)


--> X
Training until validation scores don't improve for 60 rounds
[40]	training's l1: 0.608243	training's rmse: 1.69922	valid_1's l1: 0.618185	valid_1's rmse: 1.73927
[80]	training's l1: 0.570354	training's rmse: 1.

Did not meet early stopping. Best iteration is:
[100]	training's l1: 0.519616	training's rmse: 1.41431	valid_1's l1: 0.534958	valid_1's rmse: 1.48254


elapsed: 1.85467 min

 ******************* SHIFT 24 2020-09-10 01:43:18 ***********

df.shape: (4139582, 42)


--> X
Training until validation scores don't improve for 60 rounds
[40]	training's l1: 0.486322	training's rmse: 1.32617	valid_1's l1: 0.494493	valid_1's rmse: 1.37082
[80]	training's l1: 0.462866	training's rmse: 1.24498	valid_1's l1: 0.475256	valid_1's rmse: 1.30592
Did not meet early stopping. Best iteration is:
[100]	training's l1: 0.454841	training's rmse: 1.22395	valid_1's l1: 0.469007	valid_1's rmse: 1.29098


--> Y
Training until validation scores don't improve for 60 rounds
[40]	training's l1: 0.568633	training's rmse: 1.57282	valid_1's l1: 0.580051	valid_1's rmse: 1.6241
[80]	training's l1: 0.533855	training's rmse: 1.44473	valid_1's l1: 0.549677	valid_1's rmse: 1.51474
Did not meet early stopping. Best iteration is:


Training until validation scores don't improve for 60 rounds
[40]	training's l1: 0.387699	training's rmse: 1.05894	valid_1's l1: 0.390721	valid_1's rmse: 1.07406
[80]	training's l1: 0.36745	training's rmse: 0.989396	valid_1's l1: 0.372358	valid_1's rmse: 1.0129
Did not meet early stopping. Best iteration is:
[100]	training's l1: 0.361787	training's rmse: 0.969595	valid_1's l1: 0.367775	valid_1's rmse: 0.996887


elapsed: 2.10880 min

 ******************* SHIFT 15 2020-09-10 02:01:01 ***********

df.shape: (5115979, 42)


--> X
Training until validation scores don't improve for 60 rounds
[40]	training's l1: 0.32214	training's rmse: 0.853925	valid_1's l1: 0.326127	valid_1's rmse: 0.876814
[80]	training's l1: 0.312226	training's rmse: 0.810789	valid_1's l1: 0.317966	valid_1's rmse: 0.840881
Did not meet early stopping. Best iteration is:
[100]	training's l1: 0.307885	training's rmse: 0.792572	valid_1's l1: 0.314299	valid_1's rmse: 0.825564


--> Y
Training until validation scores don't im

df.shape: (6833344, 42)


--> X
Training until validation scores don't improve for 60 rounds
[40]	training's l1: 0.196028	training's rmse: 0.480523	valid_1's l1: 0.197984	valid_1's rmse: 0.488858
[80]	training's l1: 0.192027	training's rmse: 0.46142	valid_1's l1: 0.194649	valid_1's rmse: 0.473037
Did not meet early stopping. Best iteration is:
[100]	training's l1: 0.191004	training's rmse: 0.455379	valid_1's l1: 0.193918	valid_1's rmse: 0.468475


--> Y
Training until validation scores don't improve for 60 rounds
[40]	training's l1: 0.220376	training's rmse: 0.552069	valid_1's l1: 0.222662	valid_1's rmse: 0.561401
[80]	training's l1: 0.211409	training's rmse: 0.521152	valid_1's l1: 0.214354	valid_1's rmse: 0.533527
Did not meet early stopping. Best iteration is:
[100]	training's l1: 0.209761	training's rmse: 0.513909	valid_1's l1: 0.213026	valid_1's rmse: 0.527639


elapsed: 2.65637 min

 ******************* SHIFT 06 2020-09-10 02:22:18 ***********

df.shape: (7129324, 42)


--> X
Trai

# Loading the test set as CSV

In [18]:
# Here, I'm gonna load the test, it contains `71122` rows as expected
df = pd.read_csv("../input/lyft-test-set-as-csv/Lyft_test_set.csv")
print("df.shape:", df.shape)
df.head(10)

df.shape: (71122, 45)


,frame_index_interval_start,frame_index_interval_end,host,start_time,end_time,scene_db_id,timestamp,agent_index_interval_start,agent_index_interval_end,traffic_light_faces_index_interval_start,traffic_light_faces_index_interval_end,ego_translation_x,ego_translation_y,ego_translation_z,ego_rotation_xx,ego_rotation_xy,ego_rotation_xz,ego_rotation_yx,ego_rotation_yy,ego_rotation_yz,ego_rotation_zx,ego_rotation_zy,ego_rotation_zz,frame_db_id,frame_rank,nagents,nlights,centroid_x,centroid_y,extent_x,extent_y,extent_z,yaw,velocity_x,velocity_y,track_id,label_probabilities_PERCEPTION_LABEL_UNKNOWN,label_probabilities_PERCEPTION_LABEL_CAR,label_probabilities_PERCEPTION_LABEL_CYCLIST,label_probabilities_PERCEPTION_LABEL_PEDESTRIAN,agent_db_id,nframes,centroid_xs,centroid_ys,yaw_s
0,0,100,host-a118,1578605997692823296,1578606022692823296,0,1578606007801600134,8302,8385,48,54,536.820496,-2387.457275,288.328339,0.440643,-0.897361,0.024005,0.897383,0.441029,0.014044,-0.023190,0.015353,0.999613,99,99,83,6,540.265198,-2399.707275,4.566169,2.115530,1.517801,2.021556,-0.653267,1.878533,2,0.0,1.0,0.0,0.0,0,1,540.265198,-2399.707275,2.021556
1,100,200,host-a118,1578606022692823296,1578606047692823296,1,1578606032802467516,16433,16522,609,615,624.529236,-2269.654053,285.190308,0.549425,-0.834528,0.041163,0.835520,0.548374,-0.034540,0.006252,0.053369,0.998555,199,99,89,6,611.142456,-2279.240479,4.403694,1.703298,1.464434,-2.121913,-6.067945,-9.134871,4,0.0,1.0,0.0,0.0,1,1,611.142456,-2279.240479,-2.121913
2,100,200,host-a118,1578606022692823296,1578606047692823296,1,1578606032802467516,16433,16522,609,615,624.529236,-2269.654053,285.190308,0.549425,-0.834528,0.041163,0.835520,0.548374,-0.034540,0.006252,0.053369,0.998555,199,99,89,6,618.165710,-2279.276123,3.317446,2.120635,1.576786,0.990170,1.571839,2.408892,5,0.0,1.0,0.0,0.0,2,1,618.165710,-2279.276123,0.990170
3,100,200,host-a118,1578606022692823296,1578606047692823296,1,1578606032802467516,16433,16522,609,615,624.529236,-2269.654053,285.190308,0.549425,-0.834528,0.041163,0.835520,0.548374,-0.034540,0.006252,0.053369,0.998555,199,99,89,6,640.655945,-2233.323730,3.985592,2.013386,1.448210,-2.153157,-3.022248,-4.823001,81,0.0,1.0,0.0,0.0,3,1,640.655945,-2233.323730,-2.153157
4,100,200,host-a118,1578606022692823296,1578606047692823296,1,1578606032802467516,16433,16522,609,615,624.529236,-2269.654053,285.190308,0.549425,-0.834528,0.041163,0.835520,0.548374,-0.034540,0.006252,0.053369,0.998555,199,99,89,6,597.227966,-2236.942627,4.591854,2.620991,2.097353,-0.571951,-0.196070,-0.110046,130,0.0,1.0,0.0,0.0,4,1,597.227966,-2236.942627,-0.571951
5,200,300,host-a118,1578606047692823296,1578606072692823296,2,1578606057802432986,24831,24926,615,615,780.240417,-2032.252686,293.688660,0.536894,-0.839494,0.083634,0.840752,0.540624,0.029371,-0.069871,0.054546,0.996064,299,99,95,0,765.511963,-2054.821533,2.404521,2.111213,1.467414,1.007352,7.138762,10.837037,1,0.0,1.0,0.0,0.0,5,1,765.511963,-2054.821533,1.007352
6,300,400,host-a118,1578606072692823296,1578606097692823296,3,1578606082801997166,31374,31419,615,615,963.404419,-1752.821533,269.857697,0.538929,-0.840125,0.061198,0.841174,0.540593,0.013610,-0.044517,0.044143,0.998033,399,99,45,0,950.780701,-1772.435181,2.932318,2.122027,1.706450,1.000485,7.356675,11.042892,1,0.0,1.0,0.0,0.0,6,1,950.780701,-1772.435181,1.000485
7,300,400,host-a118,1578606072692823296,1578606097692823296,3,1578606082801997166,31374,31419,615,615,963.404419,-1752.821533,269.857697,0.538929,-0.840125,0.061198,0.841174,0.540593,0.013610,-0.044517,0.044143,0.998033,399,99,45,0,973.369202,-1748.802002,5.550737,2.023970,2.039209,2.279940,-0.055759,-0.040675,707,0.0,1.0,0.0,0.0,7,1,973.369202,-1748.802002,2.279940
8,400,500,host-a118,1578606097692823296,1578606122692823296,4,1578606107802708166,38282,38341,615,615,938.855713,-1526.349243,267.135193,-0.849012,-0.528230,0.012313,0.528097,-0.847585,0.052084,-0.017076,0.050723,0.998567,499,99,59,0,950.528564,-1533.730347,3.302427,1.962232,1.54197

# Loading the LGBM models

In [19]:
def get_model_name(filename):
    return re.search("^(lgbm_[x,y]_shift_\d+)", filename).group(1)

In [20]:
def get_models(path):
    models = {}
    path = Path(path)
    for model in path.glob("lgbm*"):
        model_name = get_model_name(model.stem)
        shift = int(model_name.split("shift_")[1])
        meta = path.joinpath("meta_shift_{:02d}.json".format(shift))
        with meta.open() as f:
            train_cols = json.load(f)["TRAIN_COLS"]
        models[model_name] = {"model": model.as_posix(), "train_cols": train_cols}
    return models

In [21]:
models = get_models("./models/lyft_lgbm_model")
len(models)

100

In [22]:
next(iter(models.items()))

('lgbm_y_shift_16',
 {'model': 'models/lyft_lgbm_model/lgbm_y_shift_16.bin',
  'train_cols': ['ego_translation_x',
   'ego_translation_y',
   'ego_translation_z',
   'ego_rotation_xx',
   'ego_rotation_xy',
   'ego_rotation_xz',
   'ego_rotation_yx',
   'ego_rotation_yy',
   'ego_rotation_yz',
   'ego_rotation_zx',
   'ego_rotation_zy',
   'ego_rotation_zz',
   'extent_x',
   'extent_y',
   'extent_z',
   'velocity_x',
   'velocity_y',
   'label_probabilities_PERCEPTION_LABEL_UNKNOWN',
   'label_probabilities_PERCEPTION_LABEL_CAR',
   'label_probabilities_PERCEPTION_LABEL_CYCLIST',
   'label_probabilities_PERCEPTION_LABEL_PEDESTRIAN',
   'yaw',
   'nagents',
   'nlights',
   'centroid_xs',
   'centroid_ys']})

## Make prediction for the test set

In [23]:
def make_colnames():
    xcols = ["coord_x{}{}".format(step, rank) for step in range(3) for rank in range(50)]
    ycols = ["coord_y{}{}".format(step, rank) for step in range(3) for rank in range(50)]
    cols = ["timestamp", "track_id"] + ["conf_0", "conf_1", "conf_2"] + list(it.chain(*zip(xcols, ycols)))
    return cols

In [24]:
def predict(models, df):
    sub = np.empty((len(df), 305))
    sub.fill(np.nan)
    sub = pd.DataFrame(sub, columns = make_colnames())
    sub[["timestamp", "track_id"]] = df[["timestamp", "track_id"]]
    sub["conf_0"] = 1.0
    
    for shift in range(1, 51):
        for suffix in ["x", "y"]:
            model_info = models["lgbm_{}_shift_{:02d}".format(suffix, shift)]
                
            model = lgb.Booster(model_file= model_info["model"])
            pred = model.predict(df[model_info["train_cols"]])
            
            sub["coord_{}0{}".format(suffix, shift-1)] = pred

        if not shift%10:
            print("shift: {}".format(shift))
    
    sub.fillna(0., inplace=True)
    
    return sub

In [25]:
sub = predict(models, df)

shift: 10
shift: 20
shift: 30
shift: 40
shift: 50


In [26]:
sub.iloc[:50, :105]

,timestamp,track_id,conf_0,conf_1,conf_2,coord_x00,coord_y00,coord_x01,coord_y01,coord_x02,coord_y02,coord_x03,coord_y03,coord_x04,coord_y04,coord_x05,coord_y05,coord_x06,coord_y06,coord_x07,coord_y07,coord_x08,coord_y08,coord_x09,coord_y09,coord_x010,coord_y010,coord_x011,coord_y011,coord_x012,coord_y012,coord_x013,coord_y013,coord_x014,coord_y014,coord_x015,coord_y015,coord_x016,coord_y016,coord_x017,coord_y017,coord_x018,coord_y018,coord_x019,coord_y019,coord_x020,coord_y020,coord_x021,coord_y021,coord_x022,coord_y022,coord_x023,coord_y023,coord_x024,coord_y024,coord_x025,coord_y025,coord_x026,coord_y026,coord_x027,coord_y027,coord_x028,coord_y028,coord_x029,coord_y029,coord_x030,coord_y030,coord_x031,coord_y031,coord_x032,coord_y032,coord_x033,coord_y033,coord_x034,coord_y034,coord_x035,coord_y035,coord_x036,coord_y036,coord_x037,coord_y037,coord_x038,coord_y038,coord_x039,coord_y039,coord_x040,coord_y040,coord_x041,coord_y041,coord_x042,coord_y042,coord_x043,coord_y043,coord_x044,coord_y044,coord_x045,coord_y045,coord_x046,coord_y046,coord_x047,coord_y047,coord_x048,coord_y048,coord_x049,coord_y049
0,1578606007801600134,2,1.0,0.0,0.0,-0.048895,0.179803,-0.045670,0.377115,-0.144884,0.515959,-0.065796,0.748473,-0.195980,0.739119,-0.373246,0.928482,-0.306198,1.383150,-0.286839,2.305614,-0.362774,1.866656,-0.220183,1.432894,-0.359163,1.811864,-0.554155,2.355518,-0.550837,2.288190,-0.328491,2.730840,0.354419,2.822388,-0.646804,3.234439,-1.738817,4.780672,-1.394794,3.591307,-1.008364,5.347291,-0.989527,2.209443,0.187658,3.373502,6.102296,2.852093,-1.764036,2.984548,3.058961,3.463700,-0.827757,3.941880,-2.393538,2.086826,1.850307,2.830377,2.408549,6.251110,-1.260731,7.029381,-0.491816,8.195442,-0.797193,7.452780,1.817005,7.988579,-0.648016,5.525930,3.179374,5.762448,1.020652,10.172624,1.060045,7.054455,9.059905,4.724983,12.980371,6.848873,1.979442,7.644498,6.351219,7.972337,-0.329545,2.410224,4.462005,11.283508,2.842552,5.895009,7.557912,9.672128,3.381531,19.037956,2.893692,8.829442,13.438974,6.199169,8.490320,-0.703562,2.064135,16.185820,-0.977369,8.780082
1,1578606032802467516,4,1.0,0.0,0.0,-0.559025,-0.857375,-1.159505,-2.110268,-1.700762,-2.918715,-2.196199,-4.092315,-2.825352,-4.597891,-3.576659,-7.405145,-4.249512,-5.925467,-4.496860,-6.503882,-4.775790,-9.280376,-6.081302,-10.384481,-6.236717,-9.952437,-6.560545,-9.297873,-7.608847,-14.348619,-7.862379,-13.522077,-6.714513,-14.877366,-7.175815,-14.171018,-7.123585,-22.872330,-7.455385,-20.299564,-10.350966,-17.178580,-10.720386,-23.931099,-11.133752,-18.604854,-10.794114,-20.655236,-12.146930,-24.623833,-10.824507,-25.649701,-12.227751,-21.167439,-12.746836,-22.951760,-14.814852,-19.821964,-9.055152,-19.225835,-15.949231,-26.951203,-17.241254,-34.361443,-15.366002,-17.342000,-16.607844,-28.909818,-18.099169,-36.152203,-13.420998,-38.131996,-21.026717,-47.437750,-12.211731,-27.532714,-13.827487,-26.175475,-16.998927,-26.518629,-9.436468,-38.918101,-17.043013,-44.908545,-15.526630,-28.522702,-9.374104,-45.352994,-17.359855,-20.944808,-18.081517,-41.589194,-6.641307,-35.227119,-17.728567,-37.454451,2.645494,-54.502114,-27.496160,-36.881930,-12.326400,-30.117283,-11.458929,-23.084154
2,1578606032802467516,5,1.0,0.0,0.0,0.122341,0.216781,0.208180,0.404829,0.403633,0.783736,0.724060,0.669929,0.551755,1.060938,0.920436,1.576718,0.981454,1.488150,0.969786,1.846385,1.258034,1.020465,1.211023,2.171476,1.789011,2.230304,1.397672,3.129937,1.770164,2.059197,3.048321,2.022437,1.831620,2.084304,3.194683,2.300816,2.940655,4.131199,3.037212,5.028245,3.497013,2.388504,2.620856,2.559893,4.686587,6.345338,4.504562,1.865020,3.928072,5.593215,3.774285,5.801060,2.918834,3.107344,3.182279,5.506003,2.315717,5.620843,3.279808,6.429001,2.352812,5.809637,3.532318,4.298925,5.279031,5.793444,5.295102,7.068222,4.432748,3.057317,8.558746,3.404915,5.077504,5.358550,5.244388,3.197626,10.228386,5.234457,3.955007,7.012596,2.123226,5.732590,3.697463,16.097822,0.186766,8.198029,3.949363,6.457021,5.222399,16.7

In [27]:
sub.to_csv("submission.csv", index=False)